In [33]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#descobrir o nome da pasta

/kaggle/input/datasets/guilhermedemoura/desafio-ligia/train.csv
/kaggle/input/ligia-machine-learning/train.csv
/kaggle/input/ligia-machine-learning/test.csv


In [34]:
import pandas as pd

df_train = pd.read_csv('/kaggle/input/ligia-machine-learning/train.csv') # Carrega o arquivo CSV para dentro da variável

df_train.head() # Mostra as 5 primeiras linhas da tabela

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,265519,161919.0,1.946747,-0.752526,-1.355130,-0.661630,1.502822,4.024933,-1.479661,1.139880,...,0.076197,0.297537,0.307915,0.690980,-0.350316,-0.388907,0.077641,-0.032248,7.32,0
1,180306,124477.0,2.035149,-0.048880,-3.058693,0.247945,2.943487,3.298697,-0.002192,0.674782,...,0.038628,0.228197,0.035542,0.707090,0.512885,-0.471198,0.002520,-0.069002,2.99,0
2,42665,41191.0,-0.991920,0.603193,0.711976,-0.992425,-0.825838,1.956261,-2.212603,-5.037523,...,-2.798352,0.109526,-0.436530,-0.932803,0.826684,0.913773,0.038049,0.185340,175.10,0
3,198724,132624.0,2.285718,-1.500239,-0.747565,-1.668119,-1.394143,-0.350339,-1.427984,0.010010,...,-0.139670,0.077013,0.208310,-0.538236,-0.278032,-0.162068,0.018045,-0.063005,6.10,0
4,82326,59359.0,-0.448747,-1.011440,0.115903,-3.454854,0.715771,-0.147490,0.504347,-0.113817,...,-0.243245,-0.173298,-0.006692,-1.362383,-0.292234,-0.144622,-0.032580,-0.064194,86.10,0


In [35]:
#mostra o tipo das colunas e se falta informação
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227845 entries, 0 to 227844
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      227845 non-null  int64  
 1   Time    227845 non-null  float64
 2   V1      227845 non-null  float64
 3   V2      227845 non-null  float64
 4   V3      227845 non-null  float64
 5   V4      227845 non-null  float64
 6   V5      227845 non-null  float64
 7   V6      227845 non-null  float64
 8   V7      227845 non-null  float64
 9   V8      227845 non-null  float64
 10  V9      227845 non-null  float64
 11  V10     227845 non-null  float64
 12  V11     227845 non-null  float64
 13  V12     227845 non-null  float64
 14  V13     227845 non-null  float64
 15  V14     227845 non-null  float64
 16  V15     227845 non-null  float64
 17  V16     227845 non-null  float64
 18  V17     227845 non-null  float64
 19  V18     227845 non-null  float64
 20  V19     227845 non-null  float64
 21  V20     22

In [36]:
# Mostra a porcentagem de cada classe (0 e 1)
print("Proporção de cada classe (0 = Normal, 1 = Fraude):")
print(df_train['Class'].value_counts(normalize=True) * 100)

Proporção de cada classe (0 = Normal, 1 = Fraude):
Class
0    99.827075
1     0.172925
Name: proportion, dtype: float64


In [37]:
from sklearn.model_selection import train_test_split

# separar o que é "Dica" (X) do que é "Resposta" (y)
# tirar a coluna 'Class' do X (porque é a resposta) e o 'id' (que não serve para prever)
X = df_train.drop(['Class', 'id'], axis=1)
y = df_train['Class']

# dividir em Treino e Validação
# vou usar 80% para a IA aprender e 20% para testarmos se ela aprendeu mesmo
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Separação concluída!")
print(f"Tamanho do treino: {X_train.shape[0]} linhas")
print(f"Tamanho da validação: {X_val.shape[0]} linhas")

Separação concluída!
Tamanho do treino: 182276 linhas
Tamanho da validação: 45569 linhas


In [38]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Criar o modelo
# 'scale_pos_weight' ajuda a IA a dar mais importância para a fraude (classe 1)
# Como tenho 99.8% de 0 e 0.17% de 1 vou dar um peso maior pra o 1
modelo_fraude = XGBClassifier(scale_pos_weight=500, random_state=42)

# Treinar 
print("Treinando o modelo... Isso pode levar alguns segundos.")
modelo_fraude.fit(X_train, y_train)

# Fazer a prova (prever a probabilidade de ser fraude no grupo de validação)
y_pred_proba = modelo_fraude.predict_proba(X_val)[:, 1]

# Calcular a nota (ROC AUC)
nota_auc = roc_auc_score(y_val, y_pred_proba)

print(f"Treino finalizado!")
print(f"nota ROC AUC foi: {nota_auc:.4f}")

Treinando o modelo... Isso pode levar alguns segundos.
Treino finalizado!
nota ROC AUC foi: 0.9719


In [39]:
from sklearn.preprocessing import RobustScaler

# Escalonando a coluna Amount (usando RobustScaler para não ligar para valores absurdos)
scaler = RobustScaler()
X_train['Amount'] = scaler.fit_transform(X_train[['Amount']])
X_val['Amount'] = scaler.transform(X_val[['Amount']])

# Criando o modelo 
# max_depth=6: permite que ele veja padrões mais complexos
# n_estimators=500: ele estuda por mais tempo
# learning_rate=0.05: ele aprende devagar para não decorar errado
modelo_turbinado = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    scale_pos_weight=500,
    random_state=42,
    tree_method='hist' # Deixa o treino mais rápido
)

print("Treinando o modelo melhorado... Isso vai demorar um pouquinho mais.")
modelo_turbinado.fit(X_train, y_train)

# Nova nota pos melhorias para ver se entro melhor no leaderboard
y_pred_proba_turbo = modelo_turbinado.predict_proba(X_val)[:, 1]
nova_auc = roc_auc_score(y_val, y_pred_proba_turbo)

print(f"Nova nota ROC AUC: {nova_auc:.5f}")

Treinando o modelo melhorado... Isso vai demorar um pouquinho mais.
Nova nota ROC AUC: 0.97431


In [40]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Criando o "Gerador de Fraudes" (SMOTE)
# Ele vai criar exemplos sintéticos da classe 1 para equilibrar o jogo
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

print(f"Antes do SMOTE: {sum(y_train==1)} fraudes")
print(f"Depois do SMOTE: {sum(y_res==1)} fraudes (Equilibrado!)")

# Modelo ainda melhor, creio eu, espero
# Vamos tirar o 'scale_pos_weight' porque agora os dados já estão equilibrados
modelo_v3 = XGBClassifier(
    n_estimators=1000, # Dobrei o tempo de estudo
    max_depth=8,       # Árvores mais profundas para captar detalhes mais sutis
    learning_rate=0.03, # Ainda mais devagar para ser super preciso
    subsample=0.8,      # Usa apenas 80% dos dados por árvore para evitar decorar
    colsample_bytree=0.8,
    random_state=42,
    tree_method='hist'
)

# Treino nos dados equilibrados
print("Treinando modelo com SMOTE... Vai ser o mais demorado agora")
modelo_v3.fit(X_res, y_res)

# Verificando a nova nota na validação 
y_pred_v3 = modelo_v3.predict_proba(X_val)[:, 1]
nota_v3 = roc_auc_score(y_val, y_pred_v3)

print(f"Resultado com SMOTE: {nota_v3:.5f}") # agora é torcer pra ficar pelo menos uns 0.98 e pouco

Antes do SMOTE: 315 fraudes
Depois do SMOTE: 181961 fraudes (Equilibrado!)
Treinando modelo com SMOTE... Vai ser o mais demorado agora
Resultado com SMOTE: 0.95972


In [41]:
#o SMOTE piorou meu modelo, vou testar outras coisas novas

from lightgbm import LGBMClassifier

# Preparando o LightGBM 
#'is_unbalance=True' para ele lidar com o desequilíbrio sozinho
lgb_model = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.02,
    max_depth=10,
    num_leaves=50,
    is_unbalance=True,
    random_state=42,
    verbosity=-1
)

print("Treinando o segundo modelo (LightGBM)...")
lgb_model.fit(X_train, y_train)

# Pegando as previsões do meu melhor XGBoost (o melhor que deu 0.9743)
y_pred_xgb = modelo_turbinado.predict_proba(X_val)[:, 1]

# Pegando as previsões do novo LightGBM
y_pred_lgb = lgb_model.predict_proba(X_val)[:, 1]

# vou dar 60% de peso para o meu melhor e 40% para o novo
y_final = (y_pred_xgb * 0.6) + (y_pred_lgb * 0.4)

nota_ensemble = roc_auc_score(y_val, y_final)
print(f"Resultado do Ensemble: {nota_ensemble:.5f}") #vem 0.99

Treinando o segundo modelo (LightGBM)...
Resultado do Ensemble: 0.97429


In [42]:
#talvez o problema não esteja no metodo que eu estou ultilizando, vou criar mais colunas para ver se ajuda a melhorar a ROC AUC
# Criando uma coluna de 'Valor Normalizado'
df_train['normAmount'] = (df_train['Amount'] - df_train['Amount'].mean()) / df_train['Amount'].std()

#Criando uma relação entre Tempo e Valor
df_train['amount_per_time'] = df_train['Amount'] / (df_train['Time'] + 1)

print("Novas colunas criadas!")

Novas colunas criadas!


In [43]:
from sklearn.ensemble import VotingClassifier

# re-dividir os dados para incluir as colunas novas
X = df_train.drop(['Class', 'id'], axis=1)
y = df_train['Class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definindo os dois melhores especialistas
xgb = XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.05, scale_pos_weight=500, random_state=42)
lgb = LGBMClassifier(n_estimators=1000, is_unbalance=True, random_state=42, verbosity=-1)

# O juiz Voting Classifier (Soft significa que ele usa as probabilidades)
juiz_ia = VotingClassifier(
    estimators=[('xgb', xgb), ('lgb', lgb)],
    voting='soft',
    weights=[1.5, 1] # Dando um pouco mais de crédito para o meu XGBoost que foi melhor
)

print("O comitê de modelos está estudando os novos dados...")
juiz_ia.fit(X_train, y_train)

# Resultado Final
y_pred_final = juiz_ia.predict_proba(X_val)[:, 1]
print(f" NOTA FINAL DO COMITÊ: {roc_auc_score(y_val, y_pred_final):.5f}") #agora vai

O comitê de modelos está estudando os novos dados...
 NOTA FINAL DO COMITÊ: 0.97164


In [44]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Definindo o plano de 5 rodadas (Folds)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

print("Iniciando Validação Cruzada (5 rodadas)...")

# Rodando o treino 5 vezes
for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    # Separando os dados da rodada atual
    Xt, Xv = X.iloc[train_index], X.iloc[val_index]
    yt, yv = y.iloc[train_index], y.iloc[val_index]
    
    # Criando e treinando o modelo (com as configurações do meu melhor modelo)
    modelo_final = XGBClassifier(
        n_estimators=700,
        max_depth=6,
        learning_rate=0.03,
        scale_pos_weight=500,
        random_state=42,
        tree_method='hist'
    )
    
    modelo_final.fit(Xt, yt)
    
    # Calculando a nota da rodada
    proba = modelo_final.predict_proba(Xv)[:, 1]
    auc = roc_auc_score(yv, proba)
    scores.append(auc)
    print(f"Rodada {i+1}: AUC = {auc:.5f}")

print(f"\n MÉDIA FINAL DA VALIDACAO CRUZADA: {np.mean(scores):.5f}") #ultima tentativa

Iniciando Validação Cruzada (5 rodadas)...
Rodada 1: AUC = 0.98972
Rodada 2: AUC = 0.96777
Rodada 3: AUC = 0.97259
Rodada 4: AUC = 0.98933
Rodada 5: AUC = 0.97567

 MÉDIA FINAL DA VALIDACAO CRUZADA: 0.97902


In [45]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

# 10 Folds em vez de 5 (Mais rodadas = mais certeza)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []

print("Iniciando Validação Cruzada de Alta Precisão (10 rodadas)...")

for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    Xt, Xv = X.iloc[train_index], X.iloc[val_index]
    yt, yv = y.iloc[train_index], y.iloc[val_index]
    
    # Configuração de ELITE:
    modelo_estavel = XGBClassifier(
        n_estimators=1000,
        learning_rate=0.01, # Muito lento pra ser ultra preciso
        max_depth=7,
        subsample=0.9,
        colsample_bytree=0.9,
        gamma=1,            # Regularização para evitar oscilação
        reg_lambda=2,       # Penaliza padrões complexos demais
        scale_pos_weight=500,
        random_state=42,
        tree_method='hist',
        n_jobs=-1          
    )
    
    modelo_estavel.fit(Xt, yt)
    
    proba = modelo_estavel.predict_proba(Xv)[:, 1]
    auc = roc_auc_score(yv, proba)
    scores.append(auc)
    print(f"Rodada {i+1}: AUC = {auc:.5f}")

print(f"\n NOVA MÉDIA: {np.mean(scores):.5f}")
print(f" MENOR NOTA: {np.min(scores):.5f} |  MAIOR NOTA: {np.max(scores):.5f}")

Iniciando Validação Cruzada de Alta Precisão (10 rodadas)...
Rodada 1: AUC = 0.99909
Rodada 2: AUC = 0.98668
Rodada 3: AUC = 0.97113
Rodada 4: AUC = 0.96454
Rodada 5: AUC = 0.96477
Rodada 6: AUC = 0.97015
Rodada 7: AUC = 0.99687
Rodada 8: AUC = 0.98999
Rodada 9: AUC = 0.98218
Rodada 10: AUC = 0.98680

 NOVA MÉDIA: 0.98122
 MENOR NOTA: 0.96454 |  MAIOR NOTA: 0.99909


In [46]:
#criando e ajeitando o df_test
df_test = pd.read_csv('/kaggle/input/ligia-machine-learning/test.csv')
df_test['normAmount'] = (df_test['Amount'] - df_test['Amount'].mean()) / df_test['Amount'].std()
df_test['amount_per_time'] = df_test['Amount'] / (df_test['Time'] + 1)
# Treino final 
modelo_final_competicao = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9,
    gamma=1,
    reg_lambda=2,
    scale_pos_weight=500,
    random_state=42,
    tree_method='hist',
    n_jobs=-1
)

print("Treinando o modelo definitivo com 100% dos dados...")
modelo_final_competicao.fit(X, y)

# Previsão no arquivo de teste oficial
X_test_final = df_test.drop(['id'], axis=1)
previsoes_finais = modelo_final_competicao.predict_proba(X_test_final)[:, 1]

#  Criando o CSV para o Kaggle
submissao_elite = pd.DataFrame({
    'id': df_test['id'],
    'Class': previsoes_finais
})

submissao_elite.to_csv('submissao_elite_final.csv', index=False)
print("ARQUIVO GERADO!")

Treinando o modelo definitivo com 100% dos dados...
ARQUIVO GERADO!


In [47]:
import numpy as np

# Lista para guardar as previsões de 3 modelos diferentes
todas_previsoes = []
seeds = [42, 123, 2026] # Sementes diferentes para diversidade

print("Iniciando o Ensemble de Elite para buscar o Top 10...")

for s in seeds:
    print(f"Treinando modelo com semente {s}...")
    modelo = XGBClassifier(
        n_estimators=1200,    # Um pouco mais de árvores
        learning_rate=0.008,  # Ainda mais lento e preciso
        max_depth=7,
        subsample=0.85,
        colsample_bytree=0.85,
        gamma=1.5,            # Aumentamos um pouco o 'freio'
        reg_lambda=3,         # Regularização mais forte
        scale_pos_weight=500,
        random_state=s,
        tree_method='hist',
        n_jobs=-1
    )
    modelo.fit(X, y)
    todas_previsoes.append(modelo.predict_proba(X_test_final)[:, 1])

# Fazendo a média das 3 tentativas
previsoes_finais_ensemble = np.mean(todas_previsoes, axis=0)

# 3. Gerando o CSV final
submissao_top10 = pd.DataFrame({
    'id': df_test['id'],
    'Class': previsoes_finais_ensemble
})

submissao_top10.to_csv('submissao_tentativa2.csv', index=False)
print("ARQUIVO GERADO!")

Iniciando o Ensemble de Elite para buscar o Top 10...
Treinando modelo com semente 42...
Treinando modelo com semente 123...
Treinando modelo com semente 2026...
ARQUIVO GERADO!


In [48]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Configuração para 5 rodadas (Folds) de alta fidelidade
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_novo = []

print(" Iniciando Validação Cruzada de Precisão Máxima...")

for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    Xt, Xv = X.iloc[train_index], X.iloc[val_index]
    yt, yv = y.iloc[train_index], y.iloc[val_index]
    
    modelo_sniper = XGBClassifier(
        n_estimators=1500,     # Mais tempo para aprender os detalhes
        learning_rate=0.005,   # Ultra lento para máxima precisão
        max_depth=6,           # Profundidade equilibrada para não decorar demais
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,         # L1: ajuda a ignorar ruídos nos dados
        reg_lambda=2,          # L2: evita que o modelo fique complexo demais
        scale_pos_weight=500,
        random_state=42,
        tree_method='hist',
        n_jobs=-1
    )
    
    modelo_sniper.fit(Xt, yt)
    
    proba = modelo_sniper.predict_proba(Xv)[:, 1]
    auc = roc_auc_score(yv, proba)
    scores_novo.append(auc)
    print(f"Rodada {i+1}: AUC = {auc:.5f}")

media_final = np.mean(scores_novo)
print(f"\nNOVA MÉDIA ESTIMADA: {media_final:.5f}")


 Iniciando Validação Cruzada de Precisão Máxima...
Rodada 1: AUC = 0.99462
Rodada 2: AUC = 0.97655
Rodada 3: AUC = 0.96988
Rodada 4: AUC = 0.99349
Rodada 5: AUC = 0.98092

NOVA MÉDIA ESTIMADA: 0.98309


In [49]:
import numpy as np

# Criando a Hora do Dia (Ciclo de 24h)
df_train['Hour'] = (df_train['Time'] // 3600) % 24
df_test['Hour'] = (df_test['Time'] // 3600) % 24

# Diferença para a média (Usando Log para estabilizar)
df_train['Log_Amount'] = np.log1p(df_train['Amount'])
df_test['Log_Amount'] = np.log1p(df_test['Amount'])

# Razão entre valor e tempo (Engenharia de Recursos Avançada)
df_train['val_time_ratio'] = df_train['Log_Amount'] / (df_train['Hour'] + 1)
df_test['val_time_ratio'] = df_test['Log_Amount'] / (df_test['Hour'] + 1)

# Atualizando o X para o treino (Garantindo que as colunas novas entrem)
X = df_train.drop(['Class', 'id'], axis=1)
y = df_train['Class']

print("Novas colunas criadas!")

Novas colunas criadas!


In [50]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

#Garantindo que o X tenha as novas colunas
X = df_train.drop(['Class', 'id'], axis=1)
y = df_train['Class']

#Configuração de 5 rodadas
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_elite = []

print(f"Iniciando teste com as novas colunas ({X.shape[1]} colunas no total)...")

for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    Xt, Xv = X.iloc[train_index], X.iloc[val_index]
    yt, yv = y.iloc[train_index], y.iloc[val_index]
    
    modelo_elite = XGBClassifier(
        n_estimators=1500,
        learning_rate=0.005,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=2,
        scale_pos_weight=500,
        random_state=42,
        tree_method='hist',
        n_jobs=-1
    )
    
    modelo_elite.fit(Xt, yt)
    
    # Previsão e Cálculo da Nota
    proba = modelo_elite.predict_proba(Xv)[:, 1]
    auc = roc_auc_score(yv, proba)
    scores_elite.append(auc)
    print(f"Rodada {i+1}: ROC AUC = {auc:.5f}")

media_atual = np.mean(scores_elite)
print(f"\nMÉDIA COM NOVAS COLUNAS: {media_atual:.5f}")


Iniciando teste com as novas colunas (35 colunas no total)...
Rodada 1: ROC AUC = 0.99441
Rodada 2: ROC AUC = 0.97780
Rodada 3: ROC AUC = 0.96999
Rodada 4: ROC AUC = 0.99294
Rodada 5: ROC AUC = 0.98090

MÉDIA COM NOVAS COLUNAS: 0.98321


In [51]:
import numpy as np

# Garantindo que o TESTE tenha as colunas
df_test['Hour'] = (df_test['Time'] // 3600) % 24
df_test['Log_Amount'] = np.log1p(df_test['Amount'])
df_test['val_time_ratio'] = df_test['Log_Amount'] / (df_test['Hour'] + 1)

# Atualizando o X_test_final com as colunas novas
X_test_final = df_test.drop(['id'], axis=1)

# Lista de Sementes (Seeds) para diversidade total
seeds = [42, 2026, 7, 13]
previsoes_acumuladas = []

print(f"Iniciando o Grande Ensemble com {X.shape[1]} colunas...")

for s in seeds:
    print(f"Treinando modelo com seed {s}...")
    modelo_vitoria = XGBClassifier(
        n_estimators=2000,
        learning_rate=0.005,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=2,
        scale_pos_weight=500,
        random_state=s,
        tree_method='hist',
        n_jobs=-1
    )
    modelo_vitoria.fit(X, y)
    
    previsoes_acumuladas.append(modelo_vitoria.predict_proba(X_test_final)[:, 1])

# Média e Geração do CSV
media_vitoria = np.mean(previsoes_acumuladas, axis=0)
submissao_final = pd.DataFrame({'id': df_test['id'], 'Class': media_vitoria})
submissao_final.to_csv('submissao_definitiva_muniz.csv', index=False)

print("ARQUIVO DEFINITIVO GERADO! Agora o Top 10 vem.")

Iniciando o Grande Ensemble com 35 colunas...
Treinando modelo com seed 42...
Treinando modelo com seed 2026...
Treinando modelo com seed 7...
Treinando modelo com seed 13...
ARQUIVO DEFINITIVO GERADO! Agora o Top 10 vem.


In [52]:
import numpy as np

# Preparação garantida do Teste
df_test = pd.read_csv('/kaggle/input/ligia-machine-learning/test.csv')
df_test['normAmount'] = (df_test['Amount'] - df_test['Amount'].mean()) / df_test['Amount'].std()
df_test['amount_per_time'] = df_test['Amount'] / (df_test['Time'] + 1)
df_test['Hour'] = (df_test['Time'] // 3600) % 24
df_test['Log_Amount'] = np.log1p(df_test['Amount'])
df_test['val_time_ratio'] = df_test['Log_Amount'] / (df_test['Hour'] + 1)

X_test_final = df_test.drop(['id'], axis=1)

# Treino com Envelopamento (Aumentando profundidade e regularização)
seeds = [42, 2026, 7]
preds_finais = []

print("Buscando o Top 10 com ajuste fino de profundidade...")

for s in seeds:
    modelo_top = XGBClassifier(
        n_estimators=1800,
        learning_rate=0.004, # Mais lento ainda para ser cirúrgico
        max_depth=8,         # Aumentamos para captar relações complexas
        subsample=0.8,
        colsample_bytree=0.7, # Reduzi para maior diversidade entre árvores
        reg_alpha=0.2,       # Mais regularização L1
        reg_lambda=3,        # Mais regularização L2
        scale_pos_weight=500,
        random_state=s,
        tree_method='hist',
        n_jobs=-1
    )
    modelo_top.fit(X, y)
    preds_finais.append(modelo_top.predict_proba(X_test_final)[:, 1])

# Média e CSV
submissao_definitiva = pd.DataFrame({
    'id': df_test['id'],
    'Class': np.mean(preds_finais, axis=0)
})
submissao_definitiva.to_csv('submissao_top10.csv', index=False)
print("ARQUIVO GERADO!")

Buscando o Top 10 com ajuste fino de profundidade...
ARQUIVO GERADO!


In [53]:
import numpy as np

# Novas sementes para garantir a média alta
seeds_gemeas = [10, 88, 2024] 
preds_gemeas = []

print("Gerando a submissão gêmea para garantir o Top 7...")

for s in seeds_gemeas:
    modelo_final_gemeo = XGBClassifier(
        n_estimators=1800,
        learning_rate=0.004,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.2,
        reg_lambda=3,
        scale_pos_weight=500,
        random_state=s,
        tree_method='hist',
        n_jobs=-1
    )
    modelo_final_gemeo.fit(X, y)
    preds_gemeas.append(modelo_final_gemeo.predict_proba(X_test_final)[:, 1])

# Média e Geração do CSV
submissao_backup = pd.DataFrame({
    'id': df_test['id'],
    'Class': np.mean(preds_gemeas, axis=0)
})

submissao_backup.to_csv('submissao_top10_v2.csv', index=False)
print("SEGUNDA SUBMISSÃO DE GIGANTE GERADA!")

Gerando a submissão gêmea para garantir o Top 7...
SEGUNDA SUBMISSÃO DE GIGANTE GERADA!


In [32]:
import joblib

# Salvando o modelo 
joblib.dump(modelo_final_gemeo, 'modelo_final_ligia.joblib')
print("Artefato do modelo gerado, arquivo para o GitHub.")

Artefato do modelo gerado, arquivo para o GitHub.
